# Preprocessing census data Canada - Example
### *Green in the city*

This notebook provides the processing of Canada's census data. This data will be analyzed at the census tract level and prepared for the Green in the city project. The notebook therefore fits within the workflow for preparing the data from the demo version. This notebook is an example. Make sure to download the files as given in the notebook. You can also download a more recent file, however, the structure of the file might differ.
This notebook can be used for **all major cities** in Canada.

This notebook extracts the needed data from the census file of a city in Canada. 
The data is cleaned and merged with a shapefile, containing the geospatial data
linked to the census data. The censusfiles have to be downloaded from 
https://www12.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E and the shapefile has to contain 
a column with the same identifiers in order to be able to merge both datasets. Download the census profile containing **tract** data.
Make sure to download a shapefile from the same year as the census data. The identifiers might be edited over the years. 

<u>Note</u>: Some code blocks need a lot of memory and should not be ran when other code blocks are still running.

## 1. Preparing the notebook
In order for this notebook to work, the packages *numpy*, *pandas* and *geopandas* have to be installed. Also the package *censusclean* must be available in the repository and the folder *Raw data* must be present. Please paste the location of the repository's main directory in the variable *path*.

In [1]:
#%% Preamble
%pip install -r requirements.txt
# Change this path to the main project folder
path = "C:/Users/sande/OneDrive - UGent/Green in the city/Green_in_the_city"
import sys
import os
os.chdir(path)
sys.path.append(path)
if os.path.exists('Data preparation/Raw data/')==False:
    print('Warning: The folder with the raw data is not present in the directory "Data preparation"')
if os.path.exists('censusclean/')==False:
    print('Warning: package "censusclean" not found.')
import pandas as pd
import numpy as np
import censusclean.censusclean as cc
from censusclean.data_cleaning_CA import clean_census_ca
from censusclean.data_cleaning_CA import reshape_census_CA
import geopandas as gpd
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Note: you may need to restart the kernel to use updated packages.


## 2. Load data
In the second step the census data is loaded. The census data is organised in a long format with a column of variable names and a column of values. The function *clean_census_ca* will read the censusdata and creates a data frame with only the censustract data. 
The function is developed for census data of 2021 and 2016. Future census data might be saved under a different format or with different column names. In that case the function will have to be modified.

Since the data contains all cencustracts in Canada the  same file can be used for different cities in Canada. The function clean_census_data helps to read the file.

**Open the file with census data by running the codeblock underneath.** This can take several minutes.

In [2]:
#%% Load census data
help(clean_census_ca)
data = clean_census_ca('Data preparation/Raw data/Canada/'
                       '98-401-X2021007_eng_CSV/98-401-X2021007_English_CSV_data.csv',
                       col_var = 'CHARACTERISTIC_NAME',
                       col_val= 'C1_COUNT_TOTAL')
data.head()

Help on function clean_census_ca in module censusclean.data_cleaning_CA:

clean_census_ca(filename, col_var, col_val, geo_level_tract='Census tract')
    Clean census data of Canada.
    
    Read the file and set to a useful format for further preprocessing for the
    purpose of the 'green in the city' project. This function might have to be adapted for
    for files after 2021.
    
    Parameters
    ----------
    filename : string
        Name and path of the census file.
    col_var : string
        Name of the column containing the variable names.
    col_val : string
        Name of the column containing the values.
    geo_level_tract : string or double, optional.
        Value which indicates census tract. The default is 'Census tract'.
    
    Returns
    -------
    data_estimates : DataFrame
        The cleaned data frame.



,tract,geo_level,geo_name,variable,value
2631,9320001.00,Census tract,9320001.00,"Population, 2021",3100.0
2632,9320001.00,Census tract,9320001.00,"Population, 2016",3101.0
2633,9320001.00,Census tract,9320001.00,"Population percentage change, 2016 to 2021",0.0
2634,9320001.00,Census tract,9320001.00,Total private dwellings,723.0
2635,9320001.00,Census tract,9320001.00,Private dwellings occupied by usual residents,660.0


In the following code blocks you will select the variables of interest from the census data and transform the data frame to a long format. The variable names can be found in the column *variable* from the variable *data*. More information about the variables can be found in following file: *'Data preparation/Raw data/Canada/98-401-X2021007_eng_CSV/98-401-X2021007_English_meta.txt'*.
For the workshop, the variables concerning population (density), age, housing, household size, income and unemployment are selected. You can edit the variable names to select other variables.

In [3]:
variable_names = pd.DataFrame(data.variable.unique())
variable_names

,0
0,"Population, 2021"
1,"Population, 2016"
2,"Population percentage change, 2016 to 2021"
3,Total private dwellings
4,Private dwellings occupied by usual residents
...,...
1296,Children eligible for instruction in the minor...
1297,Children not eligible for instruction in the m...
1298,Total - Eligibility and instruction in the min...
1299,Eligible childrenwho have been instructed in t...


In [4]:
# variables of interest
var_oi = ['Population, 2021', 'Population density per square kilometre',
       'Land area in square kilometres',
       '0 to 4 years', '5 to 9 years', '10 to 14 years',
       '15 to 19 years', '20 to 24 years',
       '25 to 29 years', '30 to 34 years', '35 to 39 years',
       '40 to 44 years', '45 to 49 years', '50 to 54 years',
       '55 to 59 years', '60 to 64 years','65 to 69 years',
       '70 to 74 years', '75 to 79 years','80 to 84 years',
       '85 to 89 years','90 to 94 years', '95 to 99 years', '100 years and over',
       'Total - Occupied private dwellings by structural type of dwelling - 100% data',
       'Single-detached house', 'Semi-detached house', 'Row house',
       'Apartment or flat in a duplex',
       'Apartment in a building that has fewer than five storeys',
       'Apartment in a building that has five or more storeys',
       'Total - Private households by household size - 100% data',
       '1 person', '2 persons', '3 persons', '4 persons',
       '5 or more persons', 'Median total income in 2020 among recipients ($)',
       'Total - Total income groups in 2020 for the population aged 15 years and'
       'over in private households - 100% data','Without total income']
# Pivot wider to selected variables
data_wide = reshape_census_CA(data, var_oi)
del data
data_wide.head()

variable,tract,0 to 4 years,1 person,10 to 14 years,100 years and over,15 to 19 years,2 persons,20 to 24 years,25 to 29 years,3 persons,...,Land area in square kilometres,Median total income in 2020 among recipients ($),Population density per square kilometre,"Population, 2021",Row house,Semi-detached house,Single-detached house,Total - Occupied private dwellings by structural type of dwelling - 100% data,Total - Private households by household size - 100% data,Without total income
0,0010001.00,60.0,200.0,100.0,0.0,90.0,245.0,75.0,95.0,140.0,...,10.15,32400.0,161.9,1643.0,0.0,130.0,445.0,705.0,705.0,60.0
1,0010002.00,150.0,795.0,155.0,5.0,140.0,775.0,230.0,275.0,260.0,...,1.96,38800.0,2266.5,4451.0,150.0,170.0,950.0,2070.0,2065.0,125.0
2,0010003.01,135.0,845.0,160.0,0.0,185.0,725.0,250.0,220.0,270.0,...,1.62,37600.0,2569.0,4151.0,140.0,120.0,640.0,2060.0,2060.0,120.0
3,0010003.02,195.0,895.0,195.0,0.0,190.0,785.0,290.0,335.0,310.0,...,1.97,38000.0,2366.7,4666.0,490.0,55.0,395.0,2270.0,2270.0,150.0
4,0010004.01,210.0,735.0,140.0,0.0,175.0,705.0,555.0,515.0,290.0,...,5.85,31400.0,690.6,4040.0,225.0,200.0,360.0,1955.0,1955.0,115.0


Some variables are not yet available in the latest census data, so they have to be extracted from the previous census data. In this example educational and unemployment data and data related to housing are not yet available in the censusdata of 2021 (16 september 2022). Hence the census data of 2016 is used. 
The same procedure is used to load the data of 2016 as for the data of 2021. 

In [5]:
#%% Education, unemployment and housing
data_2016 = clean_census_ca('Data preparation/Raw data/Canada/98-401-X2016043_'
                            'eng_CSV/98-401-X2016043_English_CSV_data.csv',
                            col_var= 'DIM: Profile of Census Tracts (2247)',
                            col_val= 'Dim: Sex (3): Member ID: [1]: Total - Sex',
                            geo_level_tract=2)

var_oi = ['Total - Highest certificate, diploma or degree for the population '
          'aged 15 years and over in private households - 25% sample data',
          'No certificate, diploma or degree',
          'Secondary (high) school diploma or equivalency certificate',
          'Postsecondary certificate, diploma or degree', 'Unemployment rate',
          'Median monthly shelter costs for owned dwellings ($)','Median value '
          'of dwellings ($)']
#%%
data_2016_wide = reshape_census_CA(data_2016, var_oi)
del data_2016

## 3. Add geographical information
In this step the census tract boundaries are loaded and merged with the census data (https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-eng.cfm?year=21).
Since the boundaries change a little over time, both boundaries of 2021 and 2016 have to be loaded and joined based on their location. The resulting geodata frame contains the tract ID's for 2021 and 2016 and the censusdata can be merged to this geodata frame based on these ID's.
Paste the path to the files in the corresponding variables.

In [6]:
filename_2021 = 'Data preparation/Raw data/Canada/lct_000b21a_e/lct_000b21a_e.shp'
filename_2016 = 'Data preparation/Raw data/Canada/lct_000b16a_e/lct_000b16a_e.shp'
#%% Load geographical data
census_tracts_2021 = gpd.read_file(filename_2021)
census_tracts_2016 = gpd.read_file(filename_2016)
census_tracts = cc.join_by_location(layer_1=census_tracts_2021,
                                    layer_2=census_tracts_2016,
                                    col_id='CTUID',
                                    lsuffix='2021', rsuffix='2016')
#%% Merge GeoDataFrame with Dataframes
census_data = census_tracts.merge(data_wide, 
                                  how = 'left', 
                                  left_on= 'CTUID_2021',
                                  right_on= 'tract')
census_data = census_data.merge(data_2016_wide, 
                                how = 'left', 
                                left_on= 'CTUID_2016',
                                right_on= 'tract')
census_data.columns

Index(['CTUID_2021', 'DGUID', 'CTNAME', 'LANDAREA', 'PRUID', 'geometry',
       'index_2016', 'CTUID_2016', 'tract_x', '0 to 4 years', '1 person',
       '10 to 14 years', '100 years and over', '15 to 19 years', '2 persons',
       '20 to 24 years', '25 to 29 years', '3 persons', '30 to 34 years',
       '35 to 39 years', '4 persons', '40 to 44 years', '45 to 49 years',
       '5 or more persons', '5 to 9 years', '50 to 54 years', '55 to 59 years',
       '60 to 64 years', '65 to 69 years', '70 to 74 years', '75 to 79 years',
       '80 to 84 years', '85 to 89 years', '90 to 94 years', '95 to 99 years',
       'Apartment in a building that has fewer than five storeys',
       'Apartment in a building that has five or more storeys',
       'Apartment or flat in a duplex', 'Land area in square kilometres',
       'Median total income in 2020 among recipients ($)',
       'Population density per square kilometre', 'Population, 2021',
       'Row house', 'Semi-detached house', 'Single-deta

## 4. Final data preprocessing
In the next to final step the variables are mutated to new variables to create a more compact data frame with mostly the same variables as the data used for the USA and Flanders. A final selection of variables is made to create a dataframe with more overview.

In [7]:
#%% Mutate variables to match information of other cities
census_data['under 10'] = (census_data[['0 to 4 years',
                                       '5 to 9 years'
                                       ]].sum(axis = 1)/
                           census_data['Population, 2021'])*100
census_data['under 15'] = (census_data[['0 to 4 years',
                                       '5 to 9 years',
                                       '10 to 14 years'
                                       ]].sum(axis = 1)/
                           census_data['Population, 2021'])*100
census_data['over 65'] = (census_data[['65 to 69 years', '70 to 74 years', 
                                       '75 to 79 years','80 to 84 years', 
                                       '85 to 89 years', '90 to 94 years', 
                                       '95 to 99 years', '100 years and over'
                                       ]].sum(axis = 1)/
                           census_data['Population, 2021'])*100
census_data['over 70'] = (census_data[[ '70 to 74 years', 
                                       '75 to 79 years','80 to 84 years', 
                                       '85 to 89 years', '90 to 94 years', 
                                       '95 to 99 years', '100 years and over'
                                       ]].sum(axis = 1)/
                           census_data['Population, 2021'])*100
census_data['over 80'] = (census_data[[ '80 to 84 years', 
                                       '85 to 89 years', '90 to 94 years', 
                                       '95 to 99 years', '100 years and over'
                                       ]].sum(axis = 1)/
                           census_data['Population, 2021'])*100
census_data['alone'] = (census_data['1 person']/
                           census_data['Population, 2021'])*100
census_data['over 80'] = (census_data[[ '80 to 84 years', 
                                       '85 to 89 years', '90 to 94 years', 
                                       '95 to 99 years', '100 years and over'
                                       ]].sum(axis = 1)/
                           census_data['Population, 2021'])*100
census_data['no diploma'] = (census_data['No certificate, diploma or degree']/
                             census_data['Total - Highest certificate, diploma'
                                         ' or degree for the population aged 15'
                                         ' years and over in private households'
                                         ' - 25% sample data'])*100
census_data['high school'] = (census_data['Secondary (high) school diploma or '
                                          'equivalency certificate']/
                             census_data['Total - Highest certificate, diploma'
                                         ' or degree for the population aged 15'
                                         ' years and over in private households'
                                         ' - 25% sample data'])*100
census_data['higher diploma'] = (census_data['Postsecondary certificate, diploma'
                                             ' or degree']/
                             census_data['Total - Highest certificate, diploma'
                                         ' or degree for the population aged 15'
                                         ' years and over in private households'
                                         ' - 25% sample data'])*100
census_data['apartments'] = (census_data.loc[:,census_data.columns
                                             .str.contains('partment')]
                                         .sum(axis=1)/
                             census_data['Total - Occupied private dwellings by'
                                         ' structural type of dwelling - 100% '
                                         'data'])*100
census_data['attached'] = (census_data['Row house']/
                             census_data['Total - Occupied private dwellings by'
                                         ' structural type of dwelling - 100% '
                                         'data'])*100
census_data['detached'] = ((census_data['Semi-detached house']+
                            census_data['Single-detached house'])/
                             census_data['Total - Occupied private dwellings by'
                                         ' structural type of dwelling - 100% '
                                         'data'])*100

var_oi = ['tract_x', 'LANDAREA', 'Population, 2021', 'Population density per '
          'square kilometre','Median total income in 2020 among recipients ($)',
          'Unemployment rate', '0 to 4 years', 'under 10', 'under 15', 'over 65',
          'over 70', 'over 80', 'alone', 'no diploma', 'high school', 'higher'
          ' diploma', 'Median value of dwellings ($)',  'apartments','attached',
          'detached','geometry']
census_data = census_data[var_oi]
census_data.columns = ['tract', 'area', 'tot_pop', 'pop_dens',
          'med_inc','unempl','under_5_y','under_10_y','under_15_y','over_65_y',
          'over_70_y','over_80_y','alone','no_dipl','high_sch','degree',
          'house_val', 'apartments','attached','detached', 'geometry']

## 5. Export census data
In the final step the data is exported as a shp-file.

In [8]:
# %% Export to shapefile
census_data.to_file('Data preparation/Raw data/Canada/Census_Canada/Census_'
                    'tracts_Canada_new.shp')